## Math

This section provides an explanation of the matrix calculations for batch gradient descent in a neural network with 
- $n$ layers
- any number of neurons per layer
- any choice of differentiable loss function

**Notes**

- $\odot$ represents elementwise multiplication

**Definitions**

   **Inputs and Layers:**
   - Batch size: $n$
   - Input features: $d_0$ (number of features in the input layer)
   - $L$: Total number of layers (including the output layer)
   - $d_l$: Number of neurons in layer $l$, for $l = 1, 2, ..., L$.

   **Weights and Biases:**
   - $\tilde{W}_l$: Weight matrix for layer $l$, including biases.

   **Activations and Pre-activations:**
   - $Z_l$: Pre-activation (weighted sum) at layer $l$.
   - $H_l$: Activation at layer $l$ (a#er applying activation function).
   - $\tilde{H}_l$: Activation at layer $l$, augmented with bias column of ones.

   **Loss Function:**
   - $\mathcal{L}$: Loss function applied to the final layer $L$ and ground truth $Y$.

**Forward Propagation**

   **Initialization:**
   - Augment the input matrix with a bias column:
     $\tilde{H}_0 = [X, 1], \quad \tilde{H}_0 \in \mathbb{R}^{n \times (d_0 + 1)}.$

   **Layer-wise Computation:**
   For each layer $l = 1, 2, ..., L$:
   1. Compute pre-activation:
      $Z_l = \tilde{H}_{l-1} \tilde{W}_l, \quad Z_l \in \mathbb{R}^{n \times d_l}.$
   2. Apply activation function $f_l$:
      $H_l = f_l(Z_l), \quad H_l \in \mathbb{R}^{n \times d_l}.$
   3. Augment $H_l$ with a bias column for subsequent layers (if $l < L$):
   4. $\tilde{H}_l = [H_l, 1], \quad \tilde{H}_l \in \mathbb{R}^{n \times (d_l + 1)}.$

   For the final layer $L$, the output is:
   $H_L = f_L(Z_L), \quad H_L \in \mathbb{R}^{n \times d_L}.$

**Compute Loss**

   The loss function $\mathcal{L}$ depends on the task and the outputs $H_L$:
   - **Classification:** Cross-entropy loss, e.g., binary or categorical.
   - **Regression:** Mean squared error or other loss functions.

   Let:
   $\mathcal{L} = \frac{1}{n} \sum_{i=1}^n \ell(H_L[i], Y[i]),$
   where $\ell$ is the loss for a single sample.
   
**Backpropagation**

   **Initialization at the Output Layer:**
   1. Compute gradient of the loss w.r.t. the output layer activation: $\frac{\partial \mathcal{L}}{\partial H_L}$ (Dimensions: $\mathbb{R}^{n \times d_L}$)
   2. Backpropagate through the activation function of the output layer: $\frac{\partial \mathcal{L}}{\partial Z_L} = \frac{\partial \mathcal{L}}{\partial H_L} \odot f_L'(Z_L)$ (Dimensions: $\mathbb{R}^{n \times d_L}$)

   **Layer-wise Backpropagation:**
   For each layer $l = L, L-1, ..., 1$:
   1. Gradient w.r.t. weights: $\frac{\partial \mathcal{L}}{\partial \tilde{W}_l} = \frac{1}{n} \tilde{H}_{l-1}^\top \frac{\partial \mathcal{L}}{\partial Z_l}$ (Dimensions: $\mathbb{R}^{(d_{l-1} + 1) \times d_l}$)
   2. Backpropagate to the previous layer’s activations (if $l > 1$): $\frac{\partial \mathcal{L}}{\partial \tilde{H}_{l-1}} = \frac{\partial \mathcal{L}}{\partial Z_l} \tilde{W}_l^\top$ (Dimensions:  $\mathbb{R}^{n \times (d_{l-1} + 1)}$)
   3. Drop the gradient of the bias term from $\frac{\partial \mathcal{L}}{\partial \tilde{H}_{l-1}}$: $\frac{\partial \mathcal{L}}{\partial H_{l-1}} = \frac{\partial \mathcal{L}}{\partial \tilde{H}_{l-1}}[:, :-1] $ (Dimensions: $\mathbb{R}^{n \times d_{l-1}}$)
   4. Backpropagate through the activation function: $\frac{\partial \mathcal{L}}{\partial Z_{l-1}} = \frac{\partial \mathcal{L}}{\partial H_{l-1}} \odot f_{l-1}'(Z_{l-1})$ (Dimensions: $\mathbb{R}^{n \times d_{l-1}}$)

Note that $H_{0}$ is the input $X$, and $\tilde{H}_0 = [X, 1]$.

In [ ]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
class ArtificialNeuralNetwork:
    def __init__(self, num_layers, input_features, layer_sizes, activations, activation_gradient, loss, loss_gradient, learning_rate):
        self.num_layers = num_layers
        self.input_features = input_features
        self.layer_sizes = layer_sizes
        w = [self.input_features] + layer_sizes
        self.W = [
            np.random.normal(0, 0.05, size=(w[i] + 1, w[i + 1])) for i in range(len(w) - 1)
        ]
        self.activations = activations
        self.activation_gradient = activation_gradient
        self.loss = loss
        self.loss_gradient = loss_gradient
        self.learning_rate = learning_rate
    
    def forward(self, X: pd.DataFrame):
        X_current = X.to_numpy()
        X_current = np.concatenate((X_current, np.ones((X_current.shape[0], 1))), axis=1)
        self.pre_activations = []
        self.post_activations = [X_current]
        for i in range(self.num_layers):
            self.pre_activations.append(X_current @ self.W[i])
            X_current = np.concatenate((
                self.activations[i](self.pre_activations[-1]), 
                np.ones((self.post_activations[-1].shape[0], 1))), 
            axis=1)
            self.post_activations.append(X_current)
        return self.post_activations[-1][:, :-1]
    
    def backward(self, H: np.ndarray, y: np.ndarray):
        dldH = self.loss_gradient(H, y)
        dldZ = dldH * self.activation_gradient[-1](self.pre_activations[-1])
        weight_gradients = []
        for i in range(1, self.num_layers + 1):
            dldW = (1 / H.shape[0]) * self.post_activations[-i - 1].T @ dldZ
            weight_gradients.append(dldW)
            if i < self.num_layers:
                dldH_1 = dldZ @ self.W[-i].T
                dldH = dldH_1[:, :-1]
                dldZ = dldH * self.activation_gradient[-i - 1](self.pre_activations[-i - 1])
        weight_gradients = weight_gradients[::-1]
        for i in range(len(self.W)):
            self.W[i] = self.W[i] - self.learning_rate * weight_gradients[i]
        return self.loss(H, y)
    
    def __train(self, X, y):
        H = self.forward(X)
        return self.backward(H, y)

    def fit(self, X, y, iterations=100):
        for i in range(iterations):
            loss = self.__train(X, y)
            print(f"iteration {i + 1}/{iterations}: Error", loss)
    
    def predict(self, X):
        return self.forward(X)

In [ ]:
iris = datasets.load_iris()

In [ ]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['variety'])
df["variety"] = df["variety"].apply(
    lambda x: np.array([1, 0, 0]) if x == "Setosa" else np.array([0, 1, 0]) if x == "Virginica" else np.array([0, 0, 1])
)

X_train, X_test, y_train, y_test = train_test_split(df, df["variety"], test_size=0.33, random_state=42)
del X_train["variety"]
del X_test["variety"]

for col in X_train.columns:
    X_train[col] = StandardScaler().fit_transform(X_train[col].to_numpy().reshape((-1, 1)))
for col in X_test.columns:
    X_test[col] = StandardScaler().fit_transform(X_test[col].to_numpy().reshape((-1, 1)))

In [ ]:
model = ArtificialNeuralNetwork(
    num_layers=2,
    input_features=4,
    layer_sizes=[5, 3],
    activations=[lambda x: 1 / (1 + np.exp(-x)) for _ in range(3)],
    activation_derivatives=[lambda x: (1 / (1 + np.exp(-x))) * (1 - (1 / (1 + np.exp(-x)))) for _ in range(3)],
    loss=lambda x, y: np.mean((x - y) ** 2),
    loss_gradient=lambda x, y: 2 * (x - y),
    learning_rate=0.1
)

model.fit(X_train, np.array(list(map(lambda x: x.tolist(), y_train.to_numpy()))), iterations=1000)

In [ ]:
test_result = model.predict(X_test)

y_test = np.array(list(map(lambda x: x.tolist(), y_test.to_numpy())))

print("accuracy score:", accuracy_score(np.argmax(test_result, axis=1), np.argmax(y_test, axis=1)))
print("confusion matrix:\n", confusion_matrix(np.argmax(test_result, axis=1), np.argmax(y_test, axis=1)))